In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np

In [2]:
client = Client("tcp://172.31.24.178:8786")
client.restart()

<Client: 'tcp://172.31.24.178:8786' processes=3 threads=6, memory=5.42 GiB>

In [3]:
ddf = dd.read_csv("user_reviews_trimmed.csv")

In [4]:
ddf = ddf[["reviewerID", "helpful", "overall", "unixReviewTime"]] 
ddf["unixReviewTime"] = dd.to_datetime(ddf["unixReviewTime"],unit='ns').dt.year

In [5]:
ddf.head()

,reviewerID,helpful,overall,unixReviewTime
0,A2T0RJ91B0PQ03,"[0, 0]",1.0,1970
1,A3TYW0XA8HSGWB,"[0, 0]",5.0,1970
2,A2CME0TQU2IVVB,"[1, 1]",5.0,1970
3,A2E5IDLX7R388S,"[0, 0]",5.0,1970
4,A3CIEMYUGV6ZMR,"[0, 0]",5.0,1970


In [6]:
%%time

ddf = ddf.set_index("reviewerID")

def get_helpful_votes(series):
    ans = 0
    for ele in series:
        ans += eval(ele)[0]
    return ans
    
def get_total_votes(series):
    ans = 0
    for ele in series:
        ans += eval(ele)[1]
    return ans

meta_df = pd.DataFrame(columns=[('overall', 'count'), ('overall', 'mean'), ('unixReviewTime', 'min'), ('helpful', 'get_helpful_votes'), ('helpful', 'get_total_votes')], dtype=int)

agg_fn = {'overall': ['count','mean'], 
          'unixReviewTime':'min', 
          'helpful':[get_helpful_votes,get_total_votes]
         }

ddf1 = ddf.map_partitions(lambda df:df.groupby('reviewerID').agg(agg_fn), meta=meta_df)

meta_df = pd.DataFrame(columns=[('overall', 'count'), ('overall', 'mean'), ('unixReviewTime', 'min'), ('helpful', 'get_helpful_votes'), ('helpful', 'get_total_votes')], dtype=int)

ddf2 = ddf1.rename(columns={('overall', 'count'): "number_products_rated", 
                                          ('overall', 'mean'): "avg_ratings",
                                          ('unixReviewTime', 'min'): "reviewing_since",
                                          ('helpful', 'get_helpful_votes'):"helpful_votes",
                                          ('helpful', 'get_total_votes'): "total_votes"
                                         })

print(f"ddf2 num partitons = {ddf2.npartitions}")

submit = ddf2.describe().round(2) 

ddf_result = submit.compute()


ddf2 num partitons = 53
CPU times: user 338 ms, sys: 35.5 ms, total: 373 ms
Wall time: 1min 46s


In [7]:
ddf_result.head()

,number_products_rated,avg_ratings,reviewing_since,helpful_votes,total_votes
count,3954028.00,3954028.00,3954028.0,3954028.00,3954028.00
mean,1.52,4.14,1970.0,3.13,4.22
std,3.14,1.24,0.0,33.00,36.68
min,1.00,1.00,1970.0,0.00,0.00
25%,1.00,4.00,1970.0,0.00,0.00
